# Imports


In [29]:
# Imports  ──────────────────────────────────────────────────────────
import os                       # ← NEW
import re                       # ← NEW  (used for safe folder names)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import cholesky, solve_triangular
from scipy.special import kv, gamma
import seaborn as sns
import pyproj
import torch
import gpytorch
from torch.optim import Adam
from gpytorch.models import ExactGP
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood


In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

import os, re, pathlib

# ── Shared image root & sanitizer ─────────────────────────────────────
IMAGES_ROOT = r"C:\ASU\Semester 2\space robotics and ai\codeyy\GP\images"
NON_STAT_KERNEL = "NonStationary"

def sanitize(txt: str) -> str:
    """safe folder/file names (letters, digits, underscore)"""
    return re.sub(r'[^0-9A-Za-z_]+', '_', str(txt))


Using device: cuda


# Load data

In [31]:
#cell 2

# Load dataset
csv_file = "C:\ASU\Semester 2\space robotics and ai\codeyy\GP\Data\dec6.csv"
data = pd.read_csv(csv_file)
print(f"Loaded {len(data)} data points from: {csv_file}")

# Display dataset info
print("First few rows of raw data:")
print(data.head())
print(f"Raw Latitude range: {data['Latitude'].min()} to {data['Latitude'].max()}")
print(f"Raw Longitude range: {data['Longitude'].min()} to {data['Longitude'].max()}")



Loaded 3989 data points from: C:\ASU\Semester 2\space robotics and ai\codeyy\GP\Data\dec6.csv
First few rows of raw data:
     Time (UTC)   Latitude   Longitude  Depth (Sonar)  Temperature (°C)    pH  \
0  1.733528e+09  33.430408 -111.928888          1.551             16.72  9.04   
1  1.733528e+09  33.430408 -111.928888          1.778             16.73  9.04   
2  1.733528e+09  33.430408 -111.928888          2.031             16.73  9.04   
3  1.733528e+09  33.430408 -111.928889          2.274             16.73  9.04   
4  1.733528e+09  33.430408 -111.928888          2.604             16.73  9.04   

   Depth (m)  Conductivity (uS/cm)  Dissolved Oxygen Saturation  \
0       0.02                  1553                        127.8   
1       0.02                  1553                        127.8   
2       0.02                  1552                        127.8   
3       0.02                  1552                        127.8   
4       0.02                  1552                      

<>:4: SyntaxWarning: invalid escape sequence '\A'
<>:4: SyntaxWarning: invalid escape sequence '\A'
C:\Users\battu\AppData\Local\Temp\ipykernel_7492\2848719260.py:4: SyntaxWarning: invalid escape sequence '\A'
  csv_file = "C:\ASU\Semester 2\space robotics and ai\codeyy\GP\Data\dec6.csv"



# Latitude/Longitude coordinates 

    (WGS84) are converted to UTM (meters) using the EPSG:32612 projection for accurate spatial mapping. 
    The `pyproj.Transformer` function applies the transformation, generating X, Y coordinates in meters. 
    A sample of the transformed data and coordinate ranges is printed for verification.


In [32]:
#cell 3

# Define coordinate reference systems (CRS)
utm_crs = pyproj.CRS("EPSG:32612")  # UTM Zone 12N for Tempe, Arizona
wgs84_crs = pyproj.CRS("EPSG:4326")  # WGS84 Latitude/Longitude

# Define transformation from WGS84 (lat/lon) to UTM (X, Y in meters)
transformer = pyproj.Transformer.from_crs(wgs84_crs, utm_crs, always_xy=True)

# Apply transformation to convert Longitude/Latitude to UTM (X, Y)
data["X_coord"], data["Y_coord"] = transformer.transform(
    data["Longitude"].values,  # Input longitude (degrees)
    data["Latitude"].values    # Input latitude (degrees)
)

# Print a sample of the transformed data
print("Sample processed data (in meters):")
print(data[["Latitude", "Longitude", "X_coord", "Y_coord"]].head())

# Print coordinate ranges after conversion
print(f"X_coord range (meters): {data['X_coord'].min()} to {data['X_coord'].max()}")
print(f"Y_coord range (meters): {data['Y_coord'].min()} to {data['Y_coord'].max()}")


Sample processed data (in meters):
    Latitude   Longitude        X_coord       Y_coord
0  33.430408 -111.928888  413649.406157  3.699389e+06
1  33.430408 -111.928888  413649.368872  3.699389e+06
2  33.430408 -111.928888  413649.378070  3.699389e+06
3  33.430408 -111.928889  413649.350180  3.699389e+06
4  33.430408 -111.928888  413649.368278  3.699389e+06
X_coord range (meters): 413595.4251371465 to 413701.0693493731
Y_coord range (meters): 3699388.488145916 to 3699507.1967617515


In [33]:
# cell 4 — Scale the UTM coords
from sklearn.preprocessing import StandardScaler

# original UTM coords are in data["X_coord"], data["Y_coord"]
scaler = StandardScaler()
utm_vals = data[['X_coord','Y_coord']].values
utm_scaled = scaler.fit_transform(utm_vals)

# store back into the DataFrame
data['X_scaled'], data['Y_scaled'] = utm_scaled[:,0], utm_scaled[:,1]

print(f"UTM coords scaled: mean={utm_scaled.mean(axis=0)}, std={utm_scaled.std(axis=0)}")


UTM coords scaled: mean=[-3.44167022e-13 -3.03300751e-12], std=[1. 1.]


In [34]:
# cell 5

# Downsample dataset to a maximum of 2000 points for efficiency
max_points_per_file = 1000000000
n = len(data)  # Total number of data points

# Randomly select up to `max_points_per_file` data points (ensuring reproducibility with random_state=42)
sampled_indices = data.sample(min(n, max_points_per_file), random_state=42).index
sampled_indices = sorted(sampled_indices)  # Sorting ensures consistency in data ordering

# Extract the target variable (Surface Temperature in °C)
target_var = "Temperature (°C)"
y = data[target_var].values  # Temperature values

# Extract _scaled_ spatial features for training
X_features = data[["X_scaled", "Y_scaled"]].values



# Select only the downsampled data points
X_train = X_features[sampled_indices]
y_train = y[sampled_indices]

# Print dataset details after downsampling
print(f"Total data points: {len(data)}, Training subset size: {X_train.shape[0]}")
print("Example training point (2D):", X_train[0])


Total data points: 3989, Training subset size: 3989
Example training point (2D): [ 0.04834709 -1.59649886]


In [35]:
# cell 6 — Re‑fit the StandardScaler on the training subset to avoid data leakage
scaler = StandardScaler()

# Grab the UTM coordinates of the sampled rows
utm_train = data.loc[sampled_indices, ['X_coord', 'Y_coord']].values
utm_train_scaled = scaler.fit_transform(utm_train)

# Overwrite the scaled columns *only* for the sampled rows
data.loc[sampled_indices, 'X_scaled'] = utm_train_scaled[:, 0]
data.loc[sampled_indices, 'Y_scaled'] = utm_train_scaled[:, 1]

# Refresh the features so X_train uses the new scaling
X_features = data[['X_scaled', 'Y_scaled']].values
X_train = X_features[sampled_indices]


In [36]:
# cell 7

# Set Gaussian Process (GP) hyperparameters

# Matérn smoothness parameter:
#   nu = 0.5  -> Exponential kernel (non-differentiable paths)
#   nu = 1.5  -> Once-differentiable paths
#   nu = 2.5  -> Twice-differentiable paths (chosen here)
#   nu → ∞    -> Equivalent to the RBF (Squared Exponential) kernel


nu = 0.5

# Signal standard deviation (sigma_f): Captures the variance of the data 
# We use the standard deviation of the training targets as a rough estimate
sigma_f = np.std(y_train)  

# Noise standard deviation (sigma_n): Accounts for observation noise
# This should be set lower if sensor measurements are very precise
sigma_n = 0.1  

# Base length scale in meters: Defines the characteristic spatial scale for the kernel
base_lengthscale_space = 5  

# Print hyperparameter values
print("\nHyperparameters:")
print(f"nu = {nu}, sigma_f = {sigma_f:.2f}, sigma_n = {sigma_n}")
print(f"base_lengthscale_space = {base_lengthscale_space} meters")

# For nonstationary kernels, allow length scales to vary across space
# We define a modulation factor (alpha) that adjusts the local length scales
alpha = 0.5  # Adjust as needed

# Compute statistical properties of training data coordinates
# mean_x, mean_y -> Mean spatial positions
mean_x = np.mean(X_train[:, 0])  
mean_y = np.mean(X_train[:, 1])  

# range_x, range_y -> Spatial range (max - min) for each coordinate
range_x = max(np.ptp(X_train[:, 0]), 1e-12)   # guard against divide‑by‑zero
range_y = max(np.ptp(X_train[:, 1]), 1e-12)




Hyperparameters:
nu = 0.5, sigma_f = 0.06, sigma_n = 0.1
base_lengthscale_space = 5 meters


In [37]:
# cell 8 — Center training targets for a zero-mean GP
y_mean = np.mean(y_train)
y_train_centered = y_train - y_mean
print(f"Training targets centered: mean={np.mean(y_train_centered):.3e}")


Training targets centered: mean=-1.480e-15


In [38]:
# cell 9

def Sigma_matrix(x):
    """
    Computes the local 2x2 covariance matrix for a given location x = [x_coord, y_coord].
    This matrix is nonstationary because the length scales vary with x.
    """
    sigma_x = base_lengthscale_space * (1 + alpha * (x[0] - mean_x) / range_x)
    sigma_y = base_lengthscale_space * (1 + alpha * (x[1] - mean_y) / range_y)
    # Set an off-diagonal term (here using a fixed correlation, e.g., 0.2)
    rho = 0.2
    return np.array([
        [sigma_x**2,       rho * sigma_x * sigma_y],
        [rho * sigma_x * sigma_y, sigma_y**2]
    ])



In [39]:
# cell 10

def matern_covariance(x, x_prime, nu=nu, sigma_f=sigma_f):
    """
    Computes the nonstationary Matérn covariance between 2D points x and x_prime.
    """
    Σ_i = Sigma_matrix(x)
    Σ_j = Sigma_matrix(x_prime)
    det_Si = np.linalg.det(Σ_i)
    det_Sj = np.linalg.det(Σ_j)
    det_half = np.linalg.det((Σ_i + Σ_j) / 2.0)
    diff = np.array(x) - np.array(x_prime)
    M = (Σ_i + Σ_j) / 2.0

    try:
        v = np.linalg.solve(M, diff)
        Q_ij = float(diff.dot(v))
    except np.linalg.LinAlgError:
        v = np.linalg.pinv(M).dot(diff)
        Q_ij = float(diff.dot(v))
        
    if Q_ij < 1e-12:
        return sigma_f**2

    prefactor = (det_Si**0.25) * (det_Sj**0.25) / (det_half**0.5)
    arg = np.sqrt(2 * nu * Q_ij)
    matern_part = (arg**nu) * kv(nu, arg)
    norm_const = 1.0 / (gamma(nu) * 2**(nu - 1))
    
    return sigma_f**2 * prefactor * norm_const * matern_part

print("\nNonstationary Matérn kernel (2D) defined.")




Nonstationary Matérn kernel (2D) defined.


In [40]:
# ── cell 11 — GPyTorch Matérn-½ helper (with correct Easting ↔ Northing axes) ──
def run_gpytorch_matern(
    coords_train,           # scaled (N,2) → [easting, northing]
    y_train,                # centred (N,)
    y_mean,
    nu, sigma_n,
    results_root, date_tag, variable,
    grid_size=50
):
    import os
    import torch
    import gpytorch
    import matplotlib.pyplot as plt
    import numpy as np

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 1) Move to GPU
    train_x = torch.as_tensor(coords_train, dtype=torch.float32, device=device)
    train_y = torch.as_tensor(y_train,      dtype=torch.float32, device=device)

    # 2) Build GP model
    class GPModel(gpytorch.models.ExactGP):
        def __init__(self, x, y, lik):
            super().__init__(x, y, lik)
            self.mean_module  = gpytorch.means.ConstantMean().to(device)
            self.covar_module = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.MaternKernel(nu=nu).to(device)
            ).to(device)
        def forward(self, x):
            return gpytorch.distributions.MultivariateNormal(
                self.mean_module(x), self.covar_module(x)
            )

    likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
    model      = GPModel(train_x, train_y, likelihood)
    model.train(); likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll       = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for _ in range(80):
        optimizer.zero_grad()
        loss = -mll(model(train_x), train_y)
        loss.backward()
        optimizer.step()

    model.eval(); likelihood.eval()

    # 3) Prediction grid (easting vs northing)
    gx = torch.linspace(train_x[:,0].min(), train_x[:,0].max(), grid_size, device=device)
    gy = torch.linspace(train_x[:,1].min(), train_x[:,1].max(), grid_size, device=device)
    Xg, Yg = torch.meshgrid(gx, gy, indexing="ij")
    grid   = torch.stack([Xg.ravel(), Yg.ravel()], 1)

    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        pred = likelihood(model(grid))

    mu  = (pred.mean + y_mean).cpu().numpy()
    var = pred.variance.cpu().numpy()
    ent = 0.5 * np.log(2 * np.pi * np.e * var)

    Zmu  = mu.reshape(grid_size, grid_size)
    Zent = ent.reshape(grid_size, grid_size)
    gx_np, gy_np = gx.cpu().numpy(), gy.cpu().numpy()

    # 4) Save into the raw results_ns tree
    out_dir = os.path.join(results_root, date_tag, variable)
    os.makedirs(out_dir, exist_ok=True)

    # Mean
    plt.figure(figsize=(6,5))
    plt.contourf(gy_np, gx_np, Zmu.T, 15, cmap='viridis')
    plt.colorbar(label=variable)
    plt.xlabel("Easting (scaled units)")
    plt.ylabel("Northing (scaled units)")
    plt.title(f"{date_tag} – {variable} mean")
    plt.savefig(os.path.join(out_dir, f"{variable}_mean.png"))
    plt.close()

    # Uncertainty (Entropy)
    plt.figure(figsize=(6,5))
    plt.contourf(gy_np, gx_np, Zent.T, 15, cmap='inferno')
    plt.colorbar(label='Entropy')
    plt.xlabel("Easting (scaled units)")
    plt.ylabel("Northing (scaled units)")
    plt.title(f"{date_tag} – {variable} uncertainty")
    plt.savefig(os.path.join(out_dir, f"{variable}_uncert.png"))
    plt.close()


In [41]:
# cell 12 — File and Sensor-Variable Setup for NS-GP

# 1) list of CSVs
files = [
    r'C:\ASU\Semester 2\space robotics and ai\codeyy\GP\data\dec6.csv',
    r'C:\ASU\Semester 2\space robotics and ai\codeyy\GP\data\dec17.csv',
    r'C:\ASU\Semester 2\space robotics and ai\codeyy\GP\data\jan31.csv',
    r'C:\ASU\Semester 2\space robotics and ai\codeyy\GP\data\feb15.csv',
    r'C:\ASU\Semester 2\space robotics and ai\codeyy\GP\data\sep19.csv',
    r'C:\ASU\Semester 2\space robotics and ai\codeyy\GP\data\oct3.csv'
]

# 2) top-level results folder
results_ns = r'C:\ASU\Semester 2\space robotics and ai\codeyy\GP\results(ns)'
os.makedirs(results_ns, exist_ok=True)

# 3) columns to ignore when picking sensor vars
non_sensor_cols = [
    'Latitude', 'Longitude', 'Time (UTC)',
    'Depth (m)', 'CDOM (ppb)', 'Turbidity (NTU)'
]

# 4) prepare lat/lon → UTM transformer and hyper-params
utm_crs   = pyproj.CRS("EPSG:32612")
wgs84_crs = pyproj.CRS("EPSG:4326")
transformer = pyproj.Transformer.from_crs(wgs84_crs, utm_crs, always_xy=True)

nu                     = 0.5
base_lengthscale_space = 5
alpha                  = 0.5
sigma_n                = 0.1


In [42]:
# ── cell 12.1 — loop over CSVs & sensor variables, calling run_gpytorch_matern ──
import re
from sklearn.preprocessing import StandardScaler

def sanitize_txt(name: str) -> str:
    return re.sub(r'[^0-9A-Za-z_]+', '_', name)

for fpath in files:
    df       = pd.read_csv(fpath)
    date_tag = os.path.splitext(os.path.basename(fpath))[0]
    print(f"\n▶︎ {date_tag}")

    # lat/lon → UTM → scaled
    xs, ys        = transformer.transform(df.Longitude, df.Latitude)
    coords_scaled = StandardScaler().fit_transform(np.vstack([xs, ys]).T)

    sensor_vars = [c for c in df.columns if c not in non_sensor_cols]
    if not sensor_vars:
        print("   (no sensor columns)")
        continue

    for var in sensor_vars:
        mask = ~np.isnan(df[var].values)
        if not mask.any():
            continue

        coords_train = coords_scaled[mask]
        y_train      = df[var].values[mask]
        y_centered   = y_train - y_train.mean()

        run_gpytorch_matern(
            coords_train = coords_train,
            y_train      = y_centered,
            y_mean       = y_train.mean(),
            nu           = nu,
            sigma_n      = sigma_n,
            results_root = results_ns,       # raw NS output
            date_tag     = date_tag,
            variable     = sanitize_txt(var),
            grid_size    = 50
        )

        print(f"   ✓ {var}")



▶︎ dec6
   ✓ Depth (Sonar)
   ✓ Temperature (°C)
   ✓ pH
   ✓ Conductivity (uS/cm)
   ✓ Dissolved Oxygen Saturation
   ✓ Dissolved Oxygen Concentration (mg/L)
   ✓ Chlorophyll (ug/L)

▶︎ dec17
   ✓ Depth (Sonar)
   ✓ Temperature (°C)
   ✓ pH
   ✓ Conductivity (uS/cm)
   ✓ Dissolved Oxygen Saturation
   ✓ Dissolved Oxygen Concentration (mg/L)
   ✓ Chlorophyll (ug/L)

▶︎ jan31
   ✓ Depth (Sonar)
   ✓ Temperature (°C)
   ✓ pH
   ✓ Conductivity (uS/cm)
   ✓ Dissolved Oxygen Saturation
   ✓ Dissolved Oxygen Concentration (mg/L)
   ✓ Chlorophyll (ug/L)

▶︎ feb15
   ✓ Depth (Sonar)
   ✓ Temperature (°C)
   ✓ pH
   ✓ Conductivity (uS/cm)
   ✓ Dissolved Oxygen Saturation
   ✓ Dissolved Oxygen Concentration (mg/L)
   ✓ Chlorophyll (ug/L)

▶︎ sep19
   ✓ Depth (Sonar)
   ✓ Temperature (°C)
   ✓ pH
   ✓ Conductivity (uS/cm)
   ✓ Dissolved Oxygen Saturation
   ✓ Dissolved Oxygen Concentration (mg/L)
   ✓ Chlorophyll (ug/L)

▶︎ oct3
   ✓ Depth (Sonar)
   ✓ Temperature (°C)
   ✓ pH
   ✓ Conductivity 